In [18]:
import random
import numpy as np
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable

In [19]:
episodes = 100

capacity = 10000
sample_size = 10

learning_rate = .00001
discount_factor = 0.9
interpolation_parameter = 1e-3

epsilon_start = 1.0
epsilon_end = 0.1
epsilon_decay_rate = 1.2 / episodes

In [20]:
env = gym.make("MountainCar-v0", render_mode=None)

state_size = env.observation_space.shape[0]
action_size = env.action_space.n

In [21]:
class Network(nn.Module):
    def __init__(self, state_size, action_size, seed=42):
        super(Network, self).__init__()
        self.seed = seed
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128,636)
        self.fc3 = nn.Linear(636,64)
        self.fc4 = nn.Linear(64, action_size)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x)

In [22]:
class Memory(object):
    def __init__(self, capacity):
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
        self.capacity = capacity
        self.memory = []

    def push(self, event):
        self.memory.append(event)
        if len(self.memory) > self.capacity:
            del self.memory[0]

    def sample(self, sample_size):
        experiences = random.sample(self.memory, sample_size)
        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
        terminations = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
        return states, next_states, actions, rewards, terminations

In [23]:
class Agent():
    def __init__(self, state_size, action_size):
        self.device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
        self.state_size = state_size
        self.action_size = action_size
        self.local_network = Network(state_size, action_size).to(self.device)
        self.target_network = Network(state_size, action_size).to(self.device)
        self.optimizer = optim.Adam(self.local_network.parameters(), lr = learning_rate)
        self.memory = Memory(capacity)
        self.t_step = 0

    def step(self, state, action, reward, next_state, terminated):
        self.memory.push((state,action,reward,next_state,terminated))
        self.t_step = (self.t_step + 1) % 4
        if self.t_step == 0:
            if len(self.memory.memory) > sample_size:
                experiences = self.memory.sample(sample_size)
                self.learn(experiences, discount_factor)

    def act(self, state, epsilon):
        st = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        self.local_network.eval()
        with torch.no_grad():
            action = self.local_network(st)
        self.local_network.train()
        if random.random() > epsilon:
            return np.argmax(action.cpu().data.numpy())
        else:
            if state[1] > 0:
                action = random.choices(np.arange(self.action_size),weights=(20,20,60))
            else:
                action = random.choices(np.arange(self.action_size),weights=(60,20,20))
            return action[0]

    def learn(self, experiences, discount_factor):
        states, next_states, actions, rewards, terminations = experiences
        next_q_targets = self.target_network(next_states).detach().max(1)[0].unsqueeze(1)
        q_targets = rewards + discount_factor * next_q_targets * (1 - terminations)
        q_expected = self.local_network(states).gather(1, actions)
        loss = F.mse_loss(q_expected, q_targets)
        loss.backward()
        self.optimizer.step()
        self.soft_update(self.local_network, self.target_network, interpolation_parameter)

    def soft_update(self, local_model, target_model, interpolation_parameter):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

In [24]:
agent = Agent(state_size,action_size)

In [25]:
epsilon = epsilon_start
best = float('-inf')

for e in range(1, episodes+1):
    terminated = False
    state,_ = env.reset()
    score = 1000

    while(not terminated):
        action = agent.act(state, epsilon)
        next_state, reward, terminated,truncated,_ = env.step(action)

        if terminated:
            reward = 10
        elif next_state[1] > 0:
            if next_state[0] < 0:
                reward = (next_state[0]*-1)*next_state[1]
        elif next_state[1] < 0 and next_state[0] < 0:
                reward = ((next_state[0]*1)*next_state[1])*0.1
        else:
            reward = reward - next_state[0]*next_state[1]
        agent.step(state, action, reward, next_state, terminated)
        state = next_state
        score += reward
    epsilon = max(0.05, epsilon - epsilon_decay_rate)
    best = max(best, score)
    if e % 2 == 0:
        print(f'Episode: {e}\tScore: {score}')
env.close()

Episode: 2	Score: 938.4923102412513
Episode: 4	Score: 992.4397852683319
Episode: 6	Score: 955.4936604694577
Episode: 8	Score: 917.8481085167886
Episode: 10	Score: 972.5270708411658
Episode: 12	Score: 976.3465154095028
Episode: 14	Score: 972.7287491121383
Episode: 16	Score: 987.2227570531769
Episode: 18	Score: 977.0112310985194
Episode: 20	Score: 891.9879282834721
Episode: 22	Score: 936.070436859604
Episode: 24	Score: 943.3632097636289
Episode: 26	Score: 962.7619553695007
Episode: 28	Score: 892.0785319104049
Episode: 30	Score: 945.3165301138082
Episode: 32	Score: 966.584064094963
Episode: 34	Score: 954.1097291244158
Episode: 36	Score: 544.4776472422303
Episode: 38	Score: 700.5710251885007
Episode: 40	Score: 965.3784841340637
Episode: 42	Score: 915.2439229832472
Episode: 44	Score: 973.8970343090976
Episode: 46	Score: 949.4939866587183
Episode: 48	Score: 966.4909778755816
Episode: 50	Score: 954.3131714000634
Episode: 52	Score: 952.2576160391859
Episode: 54	Score: 956.0094363923608
Episode

In [26]:
env = gym.make("MountainCar-v0", render_mode='human')

for i in range(1,5):
    terminated = False
    state,_ = env.reset()
    score = 1000

    while(not terminated):
        action = agent.act(state, .1)
        next_state, reward, terminated,_,_ = env.step(action)

        state = next_state
        score += reward
    print(f'Episode:{i}\tScore: {score}')
env.close()

Episode:1	Score: 851.0
Episode:2	Score: 847.0
Episode:3	Score: 845.0
Episode:4	Score: 854.0
